In [1]:
!pip install psycopg2-binary pandas sqlalchemy 

import psycopg2
import sqlalchemy
import pandas
import csv


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
# CONEXIÓN
conexion = psycopg2.connect(
        host="postgresql-bancolombia.alwaysdata.net",
        user="bancolombia",
        password="pruebatecnica",
        dbname="bancolombia_prueba")

cursor = conexion.cursor()

# FUNCIÓN PARA EJECUTAR
def execute_command(command):
    cursor.execute(command)
    return None




In [ ]:
# CREAR TABLAS
comando_crear_catalogo_activos = """
CRETE TABLE IF NOT EXISTS catalogo_activos (
    cod_activo INTEGER PRIMARY KEY,
    activo VARCHAR(100) NOT NULL
);
"""

comando_crear_catalogo_banca = """
CREATE TABLE IF NOT EXISTS catalogo_banca (
    cod_banca INTEGER PRIMARY KEY,
    banca VARCHAR(100) NOT NULL
);
"""

comando_crear_cat_perfil_riesgo = """
CREATE TABLE IF NOT EXISTS cat_perfil_riesgo (
    cod_perfil_riesgo INTEGER PRIMARY KEY,
    perfil_riesgo VARCHAR(100) NOT NULL
);
"""

comando_crear_historico_aba_macroactivos = """
CREATE TABLE IF NOT EXISTS historico_aba_macroactivos (
    ingestion_year VARCHAR(50),
    ingestion_month VARCHAR(50),
    ingestion_day VARCHAR(50),
    id_sistema_cliente INTEGER PRIMARY KEY,
    macroactivo VARCHAR(50),
    cod_activo INTEGER,
    aba REAL,
    cod_perfil_riesgo INTEGER,
    cod_banca,year INTEGER,
    month VARCHAR(50)
);
"""

comando_crear_historico_aba_usd_internacional = """
CREATE TABLE IF NOT EXISTS historico_aba_macroactivos (
    ingestion_year VARCHAR(50),
    ingestion_month VARCHAR(50),
    ingestion_day VARCHAR(50),
    id_sistema_cliente INTEGER PRIMARY KEY,
    simbol VARCHAR(50),
    cusip VARCHAR(50),
    isin VARCHAR(50),
    nombre_activo VARCHAR(50),
    cantidad INTEGER,
    valor_mercado REAL,
    fecha_vencimiento DATE,
    tasa_cupon REAL
);
"""



commands = [comando_crear_catalogo_activos,comando_crear_catalogo_banca,
            comando_crear_cat_perfil_riesgo, comando_crear_historico_aba_macroactivos,
            comando_crear_historico_aba_usd_internacional]

for command in commands:
    execute_command(command)
    
conexion.commit()


In [7]:
# POBLAR TABLAS (ARCHIVOS EN EL CODESPACE)

# Función para reutilizar código
def poblar_tabla (ruta_archivo_csv,nombre_tabla, columnas):
        archivo_csv = open(ruta_archivo_csv, 'r')

        next(archivo_csv)

        cursor.copy_from(
            archivo_csv,
            nombre_tabla,
            sep=',',
            columns= columnas
        )

        archivo_csv.close()

# Tabla catalogo_activos
ruta_catalogo_activos = 'Data/catalogo_activos.csv'
nombre_catalogo_activos = 'catalogo_activos'
columnas_catalogo_activos = ('activo', 'cod_activo')

execute_command(f"TRUNCATE TABLE {nombre_catalogo_activos};") #Eliminarla para evitar duplicar

poblar_tabla(ruta_catalogo_activos,nombre_catalogo_activos, columnas_catalogo_activos)
conexion.commit()
